## Machine Learning Study #1

네이버 영화 리뷰 데이터셋을 이용하여 긍정/부정 분류하기

- TF-IDF 방법 적용
- Konlpy 한국어 형태소 분석기를 설치하고 활용
- 단어별 긍/부정 정보를 시각화하여 확인

## TF-IDF 에 대하여

정보 검색과 텍스트 마이닝에서 문서 내 단어의 중요도를 평가하기 위한 통계적 수치이다. 단어의 빈도와 그 단어가 전체 문서 집합에서 얼마나 흔한지를 함께 고려하여 특정 단어가 문서 내에서 얼마나 중요한지를 측정한다.

- TF : 특정 단어가 문서에서 등장하는 빈도를 의미한다. 보통 단어의 빈도를 문서의 총 단어 수로 나누어 정규화한다. 예를 들어, 'apple'이라는 단어가 문서에서 3번 등장하고, 문서의 총 단어 수가 100이라면 TF는 3/100 = 0.03이 된다.

- IDF : 단어가 전체 문서 집합에서 얼마나 흔하지 않은지를 측정한다. 특정 단어가 적게 등장하는 문서일 수록 IDF 값이 높아진다. 보통 로그 스케일을 사용하여 계산한다. 예를 들어 단어 'apple'이 1000개의 문서 중에서 10개의 문서에만 등장하면, IDF는 log(1000/10) = 2가 된다.

- TF-IDF(t,d) = TF(t,d) × IDF(t)

**문서 내 단어의 중요도를 평가하는 데 유용하다는 장점이 존재한다.**

## TF-IDF 에서 log 값을 사용하는 이유

1. IDF 값의 스케일 조정:

로그를 사용하면 문서 빈도가 매우 낮거나 매우 높은 경우에도 **IDF 값이 적절한 범위 내에 위치**하게 됩니다.

예를 들어, 특정 단어가 1개의 문서에서만 등장한다면, 해당 단어의 IDF 값은 매우 높아질 것입니다. 로그를 적용하지 않으면, 이 값이 너무 커져서 다른 단어들과의 비교가 어려워집니다.로그를 적용함으로써, IDF 값이 더 적절한 크기로 축소되어 다른 단어들과 비교할 수 있는 범위 내에 위치하게 됩니다.

2. 과도한 중요도 부여 방지:

**특정 단어가 거의 모든 문서에 등장하지 않는 경우, 그 단어의 IDF 값은 매우 커질 수 있습니다.** 이는 그 단어가 중요하지 않은 문서에서도 중요하다고 잘못 평가될 위험이 있습니다.로그를 적용하면 이러한 과도한 중요도 부여를 방지할 수 있습니다.

예를 들어, 단어가 전체 문서의 0.1%에서 등장하는 경우와 0.01%에서 등장하는 경우를 비교할 때, 로그를 사용하면 두 값의 차이가 완화됩니다. 즉, 로그를 사용함으로써 IDF 값의 변동폭을 줄이고, 극단적인 값이 나오는 것을 방지하여 모델의 안정성을 높일 수 있습니다.

로그를 적용함으로써,
𝑁
𝑑
𝑓
(
𝑡
)
df(t)
N
​
  값이 매우 클 때 그 크기를 완화시켜줍니다. 예를 들어,
𝑁
N이 1,000,000이고
𝑑
𝑓
(
𝑡
)
df(t)가 1인 경우, 로그를 사용하지 않으면 IDF 값이 1,000,000이 되지만, 로그를 사용하면
log
⁡
(
1
,
000
,
000
)
≈
6
log(1,000,000)≈6이 되어 훨씬 작은 값이 됩니다. 이와 같이 로그를 사용하면 과도하게 큰 값이 나오는 것을 방지하여 더 안정적이고 균형 잡힌 TF-IDF 값을 얻을 수 있습니다.

In [3]:
import pandas as pd

pd.set_option('display.max_colwidth', None) # None으로 설정하면 최대 너비 제한 없이 전체를 보여준다. -1이라고 작성하면 오류가 나서 None으로 설정함

In [6]:
df_train = pd.read_csv('/content/data/ratings_train.txt', sep='\t') # seperate를 탭을 기준으로 함
df_train

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [7]:
df_train.info() # 3개의 열로 구성된 총 15만개의 데이터

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [8]:
df_train.dropna(inplace=True) # 결측치를 제거 해보자
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


## df_train.dropna(inplace=True)

이 옵션을 사용하면 원본 데이터프레임에서 결측값이 포함된 행을 직접 삭제하고, 결과를 새로운 데이터프레임으로 반환하지 않는다.


```
     A    B   C
0  1.0  5.0   9
1  2.0  NaN  10
2  NaN  7.0  11
3  4.0  8.0  12
```

```
     A    B  C
0  1.0  5.0  9
3  4.0  8.0 12

```

위와 같이, 결측치가 있는 행 자체가 제거된다.

In [10]:
df_test = pd.read_csv('/content/data/ratings_test.txt', sep='\t')
df_test

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


In [13]:
df_test.dropna(inplace = True) # 결측치 제거 완료
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49997 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        49997 non-null  int64 
 1   document  49997 non-null  object
 2   label     49997 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


In [14]:
X_train = df_train['document'] # 훈련 데이터셋에서 document 만 뽑아서 데이터 선언
y_train = df_train['label'] # 훈련 데이터셋에서 label 만 뽑아서 데이터 선언

X_test = df_test['document']
y_test = df_test['label']

print(X_train.shape, y_train.shape) # 데이터의 형태를 출력하는 부분. 위에서 선언한 데이터의 개수를 확인 가능
print(X_test.shape, y_test.shape)

(149995,) (149995,)
(49997,) (49997,)


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

t_vect = TfidfVectorizer()
t_vect.fit(X_train[:3]) # 예제로 3개의 텍스트만 실습하기

TfidfVectorizer()

## TfidVectorizer

텍스트 데이터를 TF-IDF 특성으로 변환하는 과정이다. 텍스트 데이터의 특성을 벡터화하여 기계 학습 모델에 입력할 수 있도록 한다.

위에서는 사이킷런 라이브러리에서 TfidVectorizer 클래스를 불러와서 텍스트 데이터를 TF-IDF 형태로 변환하게 된다.

In [16]:
t_vect.vocabulary_

{'더빙': 2,
 '진짜': 6,
 '짜증나네요': 7,
 '목소리': 3,
 '포스터보고': 9,
 '초딩영화줄': 8,
 '오버연기조차': 5,
 '가볍지': 0,
 '않구나': 4,
 '너무재밓었다그래서보는것을추천한다': 1}

In [20]:
!python --version
!pip install --upgrade JPype1
!pip install --upgrade konlpy

Python 3.10.12


In [22]:
from konlpy.tag import Kkma

kkma = Kkma()

## Kkma 클래스

Konlpy에서 제공하는 한국어 형태소 분석기 중 하나이다.

1) 형태소 분석 : 주어진 한국어 문장을 형태소 단위로 분리한다.

2) 품사 태깅 : 각 형태소의 품사를 태깅한다. (명사, 동사, 형용사 등)



```
text = "안녕하세요. 한국어 자연어 처리를 공부하는 것은 정말 재미있어요."
morphs = kkma.morphs(text)  # 형태소 분석
pos = kkma.pos(text)        # 품사 태깅

print(morphs)
print(pos)
```

```
['안녕', '하', '세요', '.', '한국어', '자연어', '처리', '를', '공부', '하', '는', '것', '은', '정말', '재미있', '어요', '.']
[('안녕', 'NNG'), ('하', 'XSV'), ('세요', 'EFN'), ('.', 'SF'), ('한국어', 'NNG'), ('자연어', 'NNG'), ('처리', 'NNG'), ('를', 'JKO'), ('공부', 'NNG'), ('하', 'XSV'), ('는', 'ETD'), ('것', 'NNB'), ('은', 'JX'), ('정말', 'MAG'), ('재미있', 'VA'), ('어요', 'EFN'), ('.', 'SF')]
```

In [23]:
X_train[0]
kkma.nouns(X_train[0]) # 명사만 태깅

['더빙', '목소리']

In [24]:
def myTokenizer(text):
  return kkma.nouns(text)

In [25]:
t_vect = TfidfVectorizer(tokenizer=myTokenizer)
t_vect.fit(X_train[:3])
t_vect.vocabulary_

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


{'더빙': 0,
 '목소리': 2,
 '흠': 17,
 '포스터': 15,
 '포스터보고': 16,
 '보고': 4,
 '초': 12,
 '초딩영화줄': 13,
 '딩': 1,
 '영화': 6,
 '줄': 11,
 '오버': 7,
 '오버연기': 8,
 '연기': 5,
 '재': 9,
 '재밓': 10,
 '밓': 3,
 '추천': 14}

In [26]:
data = "머신러닝은 재밌지만 어려울 때가 있다."
kkma.morphs(data) # 형태소 추출하기

['머신', '러닝', '은', '재밌', '지만', '어렵', 'ㄹ', '때', '가', '있', '다', '.']

In [27]:
kkma.pos(data)

[('머신', 'NNG'),
 ('러닝', 'NNG'),
 ('은', 'JX'),
 ('재밌', 'VA'),
 ('지만', 'ECE'),
 ('어렵', 'VV'),
 ('ㄹ', 'ETD'),
 ('때', 'NNG'),
 ('가', 'JKS'),
 ('있', 'VV'),
 ('다', 'EFN'),
 ('.', 'SF')]

In [29]:
kkma.tagset # 품사에 대한 안내

{'EC': '연결 어미',
 'ECD': '의존적 연결 어미',
 'ECE': '대등 연결 어미',
 'ECS': '보조적 연결 어미',
 'EF': '종결 어미',
 'EFA': '청유형 종결 어미',
 'EFI': '감탄형 종결 어미',
 'EFN': '평서형 종결 어미',
 'EFO': '명령형 종결 어미',
 'EFQ': '의문형 종결 어미',
 'EFR': '존칭형 종결 어미',
 'EP': '선어말 어미',
 'EPH': '존칭 선어말 어미',
 'EPP': '공손 선어말 어미',
 'EPT': '시제 선어말 어미',
 'ET': '전성 어미',
 'ETD': '관형형 전성 어미',
 'ETN': '명사형 전성 어미',
 'IC': '감탄사',
 'JC': '접속 조사',
 'JK': '조사',
 'JKC': '보격 조사',
 'JKG': '관형격 조사',
 'JKI': '호격 조사',
 'JKM': '부사격 조사',
 'JKO': '목적격 조사',
 'JKQ': '인용격 조사',
 'JKS': '주격 조사',
 'JX': '보조사',
 'MA': '부사',
 'MAC': '접속 부사',
 'MAG': '일반 부사',
 'MD': '관형사',
 'MDN': '수 관형사',
 'MDT': '일반 관형사',
 'NN': '명사',
 'NNB': '일반 의존 명사',
 'NNG': '보통명사',
 'NNM': '단위 의존 명사',
 'NNP': '고유명사',
 'NP': '대명사',
 'NR': '수사',
 'OH': '한자',
 'OL': '외국어',
 'ON': '숫자',
 'SE': '줄임표',
 'SF': '마침표, 물음표, 느낌표',
 'SO': '붙임표(물결,숨김,빠짐)',
 'SP': '쉼표,가운뎃점,콜론,빗금',
 'SS': '따옴표,괄호표,줄표',
 'SW': '기타기호 (논리수학기호,화폐기호)',
 'UN': '명사추정범주',
 'VA': '형용사',
 'VC': '지정사',
 'VCN': "부정 지정사, 형용사 '아니다'",
 'VC

In [30]:
df = pd.DataFrame(kkma.pos(data), columns=['morph', 'tag'])
df.set_index('tag', inplace=True)
df

,morph
tag,
NNG,머신
NNG,러닝
JX,은
VA,재밌
ECE,지만
VV,어렵
ETD,ㄹ
NNG,때
JKS,가


No charts were generated by quickchart


In [33]:
def myTokenizer2(text): # 형태소 분석 및 데이터 프레임 생성
  df = pd.DataFrame(kkma.pos(text), columns=['morph', 'tag'])
  df.set_index('tag', inplace=True)

  if ('VV' in df.index) | ('VA' in df.index) | ('VX' in df.index): # 조건문을 사용해서 동사, 형용사, 보조 용언 중 하나라도 데이터 프레임의 인덱스에 포함되어 있는지 검사
    labels = ['VV', 'VA', 'NNG']
    return df.loc[df.index.intersection(labels)].dropna()['morph'].values # 해당 부분만 선택

  else:
    return [] # 없으면 빈 리스트 제출

In [34]:
myTokenizer2(data)

array(['머신', '러닝', '때', '재밌', '어렵', '있'], dtype=object)

myTokenizer2()를 사용하면 아래와 같은 결과를 얻게 된다.

```
text = "오늘은 맑은 날씨라서 밖에 나가기 좋습니다."
tokens = myTokenizer2(text)
print(tokens)
```

```
['맑', '나', '기', '좋']

```

In [35]:
t_vect = TfidfVectorizer(tokenizer=myTokenizer2) # TfidfVectorizer 에 사용자 정의 토크나이저인 myTokenizer2 함수를 지정
t_vect.fit(X_train[:3]) # 처음에는 3개 데이터를 사용하여 t_vect 객체를 학습
t_vect.vocabulary_ # 단어 사전 생성 후 이를 기반으로 학습 진행

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


{'아': 5,
 '짜증나': 12,
 '더빙': 1,
 '목소리': 2,
 '흠': 15,
 '포스터': 14,
 '보고': 4,
 '영화': 8,
 '줄': 11,
 '오버': 9,
 '연기': 7,
 '가볍': 0,
 '재': 10,
 '추천': 13,
 '어': 6,
 '보': 3}

In [36]:
t_vect = TfidfVectorizer(tokenizer=myTokenizer2)
t_vect.fit(X_train[:10000])
t_vect.vocabulary_ # 형태소 옆 숫자들은 단어 사전을 위해 생성된 고유한 숫자임(인덱스용)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


{'아': 4513,
 '짜증나': 6898,
 '더빙': 1679,
 '목소리': 2528,
 '흠': 8398,
 '포스터': 7735,
 '보고': 3097,
 '영화': 5090,
 '줄': 6641,
 '오버': 5177,
 '연기': 4995,
 '가볍': 53,
 '재': 6115,
 '추천': 7170,
 '어': 4768,
 '보': 3094,
 '교도소': 669,
 '이야기': 5707,
 '재미': 6128,
 '점': 6303,
 '조정': 6490,
 '멀': 2398,
 '없': 4869,
 '익살': 5768,
 '스파이': 4229,
 '더': 1667,
 '맨': 2372,
 '커스틴': 7303,
 '돋보이': 1762,
 '늙': 1437,
 '보이': 3127,
 '하': 7862,
 '이쁘': 5680,
 '걸음마': 344,
 '초등학교': 7098,
 '학년': 7921,
 '생인': 3749,
 '용': 5337,
 '별': 3075,
 '반개': 2835,
 '떼': 1998,
 '아깝': 4519,
 '원작': 5441,
 '긴장감': 1006,
 '살리': 3618,
 '욕': 5328,
 '이응': 5724,
 '경': 406,
 '길': 1009,
 '우': 5352,
 '생활': 3755,
 '발': 2863,
 '납': 1236,
 '감금': 140,
 '반복': 2843,
 '이': 5620,
 '드라마': 1875,
 '가족': 88,
 '사람': 3533,
 '모': 2469,
 '엿': 5055,
 '나오': 1163,
 '낫': 1239,
 '액션': 4712,
 '안': 4612,
 '있': 5926,
 '평점': 7715,
 '우드': 5355,
 '식': 4337,
 '화려': 8244,
 '낮': 1244,
 '헐': 8110,
 '길들이': 1010,
 '나': 1128,
 '죽': 6626,
 '때': 1970,
 '눈물': 1404,
 '향수': 8084,
 '자극': 5943,
 

In [37]:
X_train = t_vect.transform(X_train[:10000]) # 학습 데이터를 TF-IDF 행렬로 변환
X_test = t_vect.transform(X_test[:10000])

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression() # 로지스틱 회귀 모델 사용
lr.fit(X_train, y_train[:10000]) # 데이터를 로지스틱 회귀 모델에 학습시키기
lr.score(X_train, y_train[:10000]) # 모델에 대한 정확도 평가

0.8282

## Logistic Regression

주로 이진 분류 문제를 해결하는 데 사용되는 선형 모델이다. 회귀 모델이지만 실제로는 입력 변수의 선형 결합을 이용하여 이산적인 출력 변수를 예측하는 분류 모델로 사용한다.

장점

- 간단하고 이해하기 쉽다.
- 계수의 해석이 비교적 쉽다.
- 계산 비용이 낮고 큰 데이터셋에서도 잘 작동한다.

단점

- 선형 결합을 사용하기 때문에 복잡한 비선형 문제에는 적합하지 않을 수 있다.
- 고차원 데이터에서는 성능이 저하될 수 있다.
- 클래스가 매우 불균형한 경우에는 조정이 필요할 수 있다.

스팸 메일 분류, 고객 이탈 예측, 질병 진단 등의 문제에서 사용된다.